<a href="https://colab.research.google.com/github/hexterisk/CT-GAN/blob/master/VBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base

## Mounting

In [0]:
# Mount Google Drive files
from google.colab import drive
drive.mount('/content/drive')

## Installs

In [0]:
!pip install plotly scikit-image dicom pydicom opencv-python==3.4.2.16 numpy Keras tensorflow opencv-contrib-python==3.4.2.16

## Imports

In [0]:
import numpy as np
import pydicom
import os
import matplotlib.pyplot as plt
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.figure_factory import create_trisurf
from plotly.graph_objs import *
import dicom
import cv2
from PIL import Image
import pandas as pd
from scipy.spatial import distance

%matplotlib inline

# Listing

In [0]:
tampered_root_path = "/content/drive/My Drive/ICSML-2019/Tampered Scans"
original_root_path = "/content/drive/My Drive/ICSML-2019/Origional Scans"

In [0]:
exp1labels = np.genfromtxt('/content/drive/My Drive/ICSML-2019/Tampered Scans/labels_exp1.csv', delimiter=',')
exp2labels = np.genfromtxt('/content/drive/My Drive/ICSML-2019/Tampered Scans/labels_exp2.csv', delimiter=',')

# Utils

## Data loaders

In [0]:
# Loads images from tampered directory. Takes in parameters of root path and the label file. Returns dictionary of tampered scans in value and 'tampered' in key.

def load_images_from_folder_tampered(root, labels):
  print("Root folder: ", root)
  category = []
  images = {}
  for i in range(1, len(labels)):
    #print(str(int(labels[i][1])), " : ", str(int(labels[i][2])))
    if((int(labels[i][3]) != 0) and (int(labels[i][4])) != 0):
      path = root + '/Experiment 1 - Blind/' + str(int(labels[i][1])) + '/' + str(int(labels[i][2])) + ".dcm"
      if(os.path.isfile(path)==True):
        img = dicom.read_file(path)
        category.append(get_lbp_image(img.pixel_array))
      else:
        path = root + '/Experiment 2 - Open/' + str(int(labels[i][1])) + '/' + str(int(labels[i][2])) + ".dcm"
        if(os.path.isfile(path)==True):
          img = dicom.read_file(path)
          category.append(get_lbp_image(img.pixel_array))
        else:
          print("FILE DOES NOT EXIST!")
    else:
      continue
  images['tampered'] = category
  return images  

In [0]:
# Loads images from original directory. Takes in parameters of root path and the label file. Returns dictionary of original scans in value and 'original' in key.

def load_images_from_folder_original(root, labels):
  print("Root folder: ", root)
  category = []
  images = {}
  for i in range(1, len(labels)):
    if((int(labels[i][3]) != 0) and (int(labels[i][4])) != 0):
      found = False
      #print(str(int(labels[i][1])), " : ", str(int(labels[i][2])))
      root_path = root + '/set1'
      for original_image_path in os.listdir(root_path):
        parsed_original_image_path = original_image_path.split(".")[-1][0:4]
        if(str(int(labels[i][1])) == parsed_original_image_path):
          path = root_path + '/' + original_image_path + '/' + str(int(labels[i][2])) + ".dcm"
          if(os.path.isfile(path)==True):
            found = True
            img = dicom.read_file(path)
            category.append(get_lbp_image(img.pixel_array))
      if(found == False):
        root_path = root + '/set2'
        for original_image_path in os.listdir(root_path):
          parsed_original_image_path = original_image_path.split(".")[-1][0:4]
          if(str(int(labels[i][1])) == parsed_original_image_path):
            path = root_path + '/' + original_image_path + '/' + str(int(labels[i][2])) + ".dcm"
            if(os.path.isfile(path)==True):
              found = True
              img = dicom.read_file(path)
              category.append(get_lbp_image(img.pixel_array))
              #np.array(img.pixel_array, dtype=np.uint8)
            else:
              print("FILE DOES NOT EXIST!")
    else:
      continue
  images['original'] = category
  return images

## Helpers

### LBP

In [0]:
def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    '''

     64 | 128 |   1
    ----------------
     32 |   0 |   2
    ----------------
     16 |   8 |   4    

    '''    
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top
    
    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val    

def show_output(output_list):
    output_list_len = len(output_list)
    figure = plt.figure(figsize=(15, 15))
    for i in range(output_list_len):
        current_dict = output_list[i]
        current_img = current_dict["img"]
        current_xlabel = current_dict["xlabel"]
        current_ylabel = current_dict["ylabel"]
        current_xtick = current_dict["xtick"]
        current_ytick = current_dict["ytick"]
        current_title = current_dict["title"]
        current_type = current_dict["type"]
        current_plot = figure.add_subplot(1, output_list_len, i+1)
        if current_type == "gray":
            current_plot.imshow(current_img, cmap = plt.get_cmap('gray'))
            current_plot.set_title(current_title)
            current_plot.set_xticks(current_xtick)
            current_plot.set_yticks(current_ytick)
            current_plot.set_xlabel(current_xlabel)
            current_plot.set_ylabel(current_ylabel)
        elif current_type == "histogram":
            current_plot.plot(current_img, color = "black")
            current_plot.set_xlim([0,260])
            current_plot.set_title(current_title)
            current_plot.set_xlabel(current_xlabel)
            current_plot.set_ylabel(current_ylabel)            
            ytick_list = [int(i) for i in current_plot.get_yticks()]
            current_plot.set_yticklabels(ytick_list,rotation = 90)

    plt.show()

def get_dicom_grayscale_cv2(data):
    pil_image = Image.fromarray(data.astype('uint8'), 'P')
    rgbimg = Image.new("RGB", pil_image.size)
    rgbimg.paste(pil_image)
    img = cv2.cvtColor(np.array(rgbimg).astype('uint8'), cv2.COLOR_BGR2GRAY)
    return img
  
def get_lbp_image(img_dicom):
    img_dicom_gray = get_dicom_grayscale_cv2(img_dicom)
    height, width = img_dicom_gray.shape
    
    img_lbp = np.zeros((height, width, 3), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
            img_lbp[i, j] = lbp_calculated_pixel(img_dicom_gray, i, j)

    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    output_list = []
    output_list.append({
        "img": img_dicom_gray,
        "xlabel": "",
        "ylabel": "",
        "xtick": [],
        "ytick": [],
        "title": "Gray Image",
        "type": "gray"        
    })
    output_list.append({
        "img": img_lbp,
        "xlabel": "",
        "ylabel": "",
        "xtick": [],
        "ytick": [],
        "title": "LBP Image",
        "type": "gray"
    })    
    # output_list.append({
    #     "img": hist_lbp,
    #     "xlabel": "Bins",
    #     "ylabel": "Number of pixels",
    #     "xtick": None,
    #     "ytick": None,
    #     "title": "Histogram(LBP)",
    #     "type": "histogram"
    # })

    return img_lbp#, output_list

### SIFT

In [0]:
# Creates descriptors using sift 
# Takes one parameter that is images dictionary
# Return an array whose first index holds the decriptor_list without an order
# And the second index holds the sift_vectors dictionary which holds the descriptors but this is seperated class by class
def sift_features(images):
    sift_vectors = {}
    descriptor_list = []
    sift = cv2.xfeatures2d.SIFT_create()
    for key,value in images.items():
        #plt.imshow(value)
        features = []
        for img in value:
            kp, des = sift.detectAndCompute(img,None)
           
            
            descriptor_list.extend(des)
            features.append(des)
        sift_vectors[key] = features
    return [descriptor_list, sift_vectors]

In [0]:
def gen_sift_features(gray_img):
    sift = cv2.xfeatures2d.SIFT_create()
    # kp is the keypoints
    #
    # desc is the SIFT descriptors, they're 128-dimensional vectors
    # that we can use for our final features
    kp, desc = sift.detectAndCompute(gray_img, None)
    return kp, desc

def show_sift_features(gray_img, color_img, kp):
    return plt.imshow(cv2.drawKeypoints(gray_img, kp, color_img.copy()))

def draw(img_lbp, img_tlbp):
    okp, odesc = gen_sift_features(img_lbp)
    o_tkp, o_tdesc = gen_sift_features(img_tlbp)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(odesc, o_tdesc)
    # Sort the matches in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    # draw the top N matches
    N_MATCHES = 100

    match_img = cv2.drawMatches(
        img_lbp, okp,
        img_tlbp, o_tkp,
        matches[:N_MATCHES], img_tlbp.copy(), flags=0)
    
    plt.figure(figsize=(12,6))
    plt.imshow(match_img);

### K-Means

In [0]:
def kplot(centroids, reduced_data):
  # Step size of the mesh. Decrease to increase the quality of the VQ.
  h = .02     # point in the mesh [x_min, x_max]x[y_min, y_max].

  # Plot the decision boundary. For that, we will assign a color to each
  x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
  y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
  xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

  plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
  # Plot the centroids as a blue X
  plt.scatter(centroids[:, 0], centroids[:, 1],
              marker='x', s=169, linewidths=3,
              color='b', zorder=10)
  plt.title('K-means clustering (PCA-reduced data)\n'
            'Centroids are marked with blue cross')
  plt.xlim(x_min, x_max)
  plt.ylim(y_min, y_max)
  plt.xticks(())
  plt.yticks(())
  plt.show()

In [0]:
# A k-means clustering algorithm who takes 2 parameter which is number 
# of cluster(k) and the other is descriptors list(unordered 1d array)
# Returns an array that holds central points.
def kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters = k, n_init=10)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_ 
    kplot(visual_words, np.array(descriptor_list))
    return visual_words

### Histogram

In [0]:
def find_index(image, center):
    count = 0
    ind = 0
    for i in range(len(center)):
        if(i == 0):
           count = distance.euclidean(image, center[i]) 
           #count = L1_dist(image, center[i])
        else:
            dist = distance.euclidean(image, center[i]) 
            #dist = L1_dist(image, center[i])
            if(dist < count):
                ind = i
                count = dist
    return ind
  
# Takes 2 parameters. The first one is a dictionary that holds the descriptors that are separated class by class 
# And the second parameter is an array that holds the central points (visual words) of the k means clustering
# Returns a dictionary that holds the histograms for each images that are separated class by class. 
def image_class(all_bovw, centers):
    dict_feature = {}
    for key,value in all_bovw.items():
        category = []
        for img in value:
            histogram = np.zeros(len(centers))
            for each_feature in img:
                ind = find_index(each_feature, centers)
                histogram[ind] += 1
            category.append(histogram)
        dict_feature[key] = category
    return dict_feature

### KNN

In [0]:
# 1-NN algorithm. We use this for predict the class of test images.
# Takes 2 parameters. images is the feature vectors of train images and tests is the feature vectors of test images
# Returns an array that holds number of test images, number of correctly predicted images and records of class based images respectively
def knn(images, tests):
    num_test = 0
    correct_predict = 0
    class_based = {}
    
    for test_key, test_val in tests.items():
        class_based[test_key] = [0, 0] # [correct, all]
        for tst in test_val:
            predict_start = 0
            #print(test_key)
            minimum = 0
            key = "a" #predicted
            for train_key, train_val in images.items():
                for train in train_val:
                    if(predict_start == 0):
                        minimum = distance.euclidean(tst, train)
                        #minimum = L1_dist(tst,train)
                        key = train_key
                        predict_start += 1
                    else:
                        dist = distance.euclidean(tst, train)
                        #dist = L1_dist(tst,train)
                        if(dist < minimum):
                            minimum = dist
                            key = train_key
            
            if(test_key == key):
                correct_predict += 1
                class_based[test_key][0] += 1
            num_test += 1
            class_based[test_key][1] += 1
            #print(minimum)
    return [num_test, correct_predict, class_based]

### Accuracy

In [0]:
# Calculates the average accuracy and class based accuracies.  
def accuracy(results):
    avg_accuracy = (results[1] / results[0]) * 100
    print("Average accuracy: %" + str(avg_accuracy))
    print("\nClass based accuracies: \n")
    for key,value in results[2].items():
        acc = (value[0] / value[1]) * 100
        print(key + " : %" + str(acc))

# Model

In [0]:
images_train = {}
images_tamp = load_images_from_folder_tampered(tampered_root_path, exp1labels)
images_orig = load_images_from_folder_original(original_root_path, exp1labels)

images_train.update(images_orig)
images_train.update(images_tamp)

print(len(images_train['tampered']))
print(len(images_train['original']))

Root folder:  /content/drive/My Drive/ICSML-2019/Tampered Scans


In [0]:
images_test = {}

images_tamp_test = load_images_from_folder_tampered(tampered_root_path, exp2labels)
images_orig_test = load_images_from_folder_original(original_root_path, exp2labels)

images_test.update(images_orig_test)
images_test.update(images_tamp_test)

print(len(images_test['tampered']))
print(len(images_test['original']))

In [0]:
sifts = sift_features(images_train)

# Takes the descriptor list which is unordered one
descriptor_list = sifts[0] 
# Takes the sift features that is seperated class by class for train data
all_bovw_feature = sifts[1] 
# Takes the sift features that is seperated class by class for test data
test_bovw_feature = sift_features(images_test)[1]

In [0]:
print("Drawing keypoints to match similar features in original and tampered images:")
draw(images_train['original'][0], images_train['tampered'][0])

In [0]:
print("SIFT descriptors are vectors of shape.")
print("Numerical representatin of SIFTs: ")
print(sifts)
print("Graphical representation of SIFTs:")
plt.imshow(np.array(sifts)[0][0].reshape(16,8), interpolation='none')

In [0]:
# Takes the central points which is visual words    
visual_words = kmeans(2, descriptor_list)

In [0]:
# Creates histograms for train data    
bovw_train = image_class(all_bovw_feature, visual_words) 
# Creates histograms for test data
bovw_test = image_class(test_bovw_feature, visual_words) 

In [0]:
# Call the knn function    
results_bowl = knn(bovw_train, bovw_test) 

In [0]:
# Calculates the accuracies and write the results to the console.       
accuracy(results_bowl)